In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [2]:
######################
# Initiate Backtest  #
######################
from lib.multitasking_lib import *
cache_type ='backtest_uni'

my_cache = cache_state('backtest_uni')

def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("log/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9', mode='full'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler"], [kite_simulator, ohlc_tick_handler])
    
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


INFO:	Cache Pointing to: backtest_uni
INFO:	Cache Pointing to: backtest_uni
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: ohlc_tick_handler


True
['ohlc_tick_handlerlive', 'ohlc_tick_handlerbacktest_web', 'live_trade_handlerlive', 'kite_simulatorbacktest_web', 'order_handlerlive', 'freedom_initfreedom']
[<Process(Process-1, started)>]


DEBUG:	kite_simulator: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	ohlc_tick_handler: 1
DEBUG:	kite_simulator: {"stock": ["TCS"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1_new", "fromDate": "2019-4-8", "toDate": "2019-4-9", "freq": "1T", "mode": "full"}
DEBUG:	Trade_init: TCS
INFO:	Reset Cache for: TCSbacktest_uni
INFO:	TCSbacktest_uni=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day'}
INFO:	Running Full Backtest
DEBUG:	ohlc_tick_handler: start
TRADE: S: EN:  : 2019-04-08 09:18 : TCS -> 2053.0
TRADE: B: EX:  : 2019-04-08 09:20 : TCS -> 2057.55
TRADE: S: EN:  : 2019-04-08 09:24 : TCS -> 2059.75
TRADE: B: EX:  : 2019-04-08 09:25 : TCS -> 2061.0
TRADE: B: EN:  : 2019-04-08 0

In [3]:
backtest_unitest(['TCS'])

In [28]:
my_cache.get('TCSbacktest_uniTICK')

'{}'

In [53]:
################################
###         Stop Trade       ###
################################

my_cache.delete('msgBufferQueue'+cache_postfix)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())

jobs
jobs = []

False
['ohlc_tick_handlerlive', 'ohlc_tick_handlerbacktest_web', 'live_trade_handlerlive', 'kite_simulatorbacktest_web', 'order_handlerlive', 'freedom_initfreedom']


In [ ]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [ ]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

In [ ]:
cache_state('backtest_uni').getValue()

In [ ]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index[-1].minute+ohlc_data_temp.index[-1].hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

#sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
#print(trade_df1)
sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
buy = (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

In [ ]:
buy = pd.DataFrame()

if buy.shape[0] > 0 and buy[-1]==True:
    print('Yeah')

## Scratchpad

In [ ]:
redis_conn.pubsub_channels()

In [ ]:
my_cache = cache_state('backtest_uni')

In [ ]:
df = my_cache.getValue()

In [ ]:
pd.set_option('display.max_rows', 1000)
my_cache.getOHLC('TCS')

In [ ]:
trades_df = my_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

In [ ]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')
live_cache.getValue()
#live_cache.getOHLC('TCS', '1D').head()

In [ ]:
from lib.data_model_lib import *
live_cache = cache_state('live')

In [ ]:
from kiteconnect import KiteConnect

In [ ]:
kite_api_key = 'b2w0sfnr1zr92nxm'

live_cache.set('KiteAPIKey',kite_api_key)

In [ ]:
live_cache.get('access_token')

In [ ]:
live_cache.get('KiteAPIKey')

In [ ]:
KiteAPIKey = live_cache.get('KiteAPIKey')
kite = KiteConnect(api_key=KiteAPIKey)
access_token = live_cache.get('access_token')
kite.set_access_token(access_token)

In [ ]:
kite.profile()

In [ ]:
instruments_df = pd.DataFrame(kite.instruments())

In [ ]:
kite.orders()

In [ ]:
kite.positions()

In [ ]:
kite.trades()

In [ ]:
instruments_df.to_hdf('data/instruments.hdf',key='instruments')

In [ ]:
instruments_df.columns

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

In [ ]:
kite.margins()

In [ ]:
live_cache.publish('live_trade_handlerlive','INIT')

In [ ]:
live_cache.publish('live_trade_handlerlive','CLOSE')

In [ ]:
live_cache.publish('live_trade_handlerlive','START')

In [ ]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'add','value':[969473, 2953217],'mode':'quote'}))

In [ ]:
import json

In [ ]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'remove','value':[969473, 2953217],'mode':'quote'}))

In [ ]:
live_cache.publish('live_trade_handlerlive','add')

In [ ]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':187.5,'qty':1}))

In [ ]:
live_cache.publish('live_trade_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':184.6,'qty':1}))

In [ ]:
json.dumps({'cmd':'buy','symbol':'WIPRO','price':187.5,'qty':1})

In [ ]:
buy_limit('WIPRO',186.5,1)

In [ ]:
sell_limit('WIPRO',187.2,1)

In [ ]:
def buy_limit(symbol, price, quantity=1): 
    pinfo("B Limit: {}[{}]=> {}".format(symbol, quantity, price))

    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_LIMIT,
                                product=kite.PRODUCT_MIS,
                                #trigger_price=round(trigger,1),
                                #stoploss=round(stoploss,1),
                                #trigger_price=round(price,1),
                                price=price,
                                variety=kite.VARIETY_REGULAR,
                                tag='freedom_v2'
                                )
        pinfo("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        pinfo("Order placement failed: {}".format(e.message))
        
def sell_limit(symbol, price, quantity=1):
    pinfo("S Limit: {}[{}]=> {}".format(symbol, quantity, price))

    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=quantity,
                            order_type=kite.ORDER_TYPE_LIMIT,
                            product=kite.PRODUCT_MIS,
                            #trigger_price=round(trigger,1),
                            #trigger_price=round(price,1),
                            price=price,
                            variety=kite.VARIETY_REGULAR)
        pinfo("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        pinfo("Order placement failed: {}".format(e.message))

In [ ]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [ ]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


In [ ]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

In [ ]:
live_cache.delete('eq_token')

In [ ]:
eq_nse.head().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

In [ ]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

In [ ]:
live_cache.hgetall('eq_token')

In [ ]:
live_cache.hget('eq_token',[2953217, 969473])

In [ ]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [ ]:
msg['data']

In [ ]:
val = json.loads(msg['data'])

In [ ]:
for data in val:
    instrument_token = data['instrument_token']
    stock_id = live_cache.hmget('eq_token',instrument_token)[0]


    ltp = data['last_price']
    #temp_df = pd.DataFrame(data={'date':[date_val],'ltp':[ltp]})
    #temp_df = temp_df.set_index('date')
    #temp_df.index = pd.to_datetime(temp_df.index)
    print(stock_id)
    print(ltp)
    hash_key = stock_id

In [ ]:
stock_id

In [ ]:
live_cache.getValue()

In [ ]:
live_cache.add('WIPRO')

In [ ]:
stock_key = 'TCS'
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')

In [ ]:
live_cache.add('TCS')

In [ ]:
live_cache.remove()

In [ ]:
live_cache.getValue()

In [ ]:
live_cache.hmget('eq_token','TCS')[0]

In [39]:
from lib.data_model_lib import *

In [40]:
live_cache = cache_state('backtest_web')

INFO:	Cache Pointing to: backtest_web


In [6]:
live_cache.hmget('eq_token','TCS')

['2953217']

In [17]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,TCS,10,1,1,0,0,0,0.0,0,0.0,...,2132.0,0.0,0.0,2266.95,2019-05-03 00:00,SCANNING,PAUSE,haikin_1_new,1D,day
1,WIPRO,10,1,1,0,0,0,0.0,0,0.0,...,290.9,0.0,0.0,299.45,2019-05-03 00:00,SCANNING,PAUSE,haikin_1_new,1D,day


In [18]:
live_cache.getOHLC('TCS')

,close,high,low,open
2019-04-23,2155.05,2165.50,2134.05,2163.00
2019-04-24,2183.45,2195.00,2156.10,2158.00
2019-04-25,2195.00,2205.00,2168.05,2175.00
2019-04-26,2238.55,2243.95,2193.15,2203.00
2019-04-30,2260.35,2266.95,2230.10,2238.55
2019-05-02,2215.40,2259.40,2211.10,2255.00
2019-05-03,2132.00,2186.05,2125.00,2185.00


In [41]:
live_cache.xrange('msgBufferQueuebacktest_uni')

[('1554714930000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2055.15}]'}),
 ('1554714965000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2055.0}]'}),
 ('1554714970000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2060.4}]'}),
 ('1554714980000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2054.8}]'}),
 ('1554714990000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2059.0}]'}),
 ('1554715025000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2057.7}]'}),
 ('1554715030000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2058.0}]'}),
 ('1554715040000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2052.85}]'}),
 ('1554715050000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2053.35}]'}),
 ('1554715085000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2053.35}]'}),
 ('1554715090000-0',
  {'data': '[{"instrument_token": 2953217, "last_price": 2055.25}